In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv("task1_data.txt")
train_labels = pd.read_csv("task1_train_label.txt", header=None, sep="\t", names=["order", "label"])
valid_labels = pd.read_csv("task1_valid_label.txt", header=None, sep="\t", names=["order", "label"])
test_query = pd.read_csv("task1_test_query.txt",header=None, names=["order"])

In [ ]:
train_df = df.merge(train_labels, on="order")
valid_df = df.merge(valid_labels, on="order")
test_df = df.merge(test_query, on="order")

In [ ]:
# Order size correlate with label?
size_label = train_df.groupby("order").size().to_frame(name="count").merge(train_labels,on="order")
correlation = size_label['count'].corr(size_label['label'])


In [ ]:
size_label.sort_values(by="count", ascending=False).head(500)

In [ ]:
corr = train_df.corr()
corr

In [ ]:
train_df["group"].hist(bins=train_df["group"].nunique())

In [ ]:
train_df[train_df["order"] == 11376]

# Create customer stats
Return rates go up when ordering different size (and group) than usual

In [ ]:
train_order_customer = train_df.groupby('order')['customer'].first().reset_index()


In [ ]:
s = ["group", "customer"]
print("_"+"_".join(s)+"_stat")

In [ ]:
def create_user_stat_dist(df, group_by_cols, agg_col, agg_func):
    suff = "_"+"_".join(group_by_cols) + "_stat"
    agg_col_stat = agg_col + suff
    
    customer_stat =  df.groupby(group_by_cols).agg({agg_col: agg_func})
    merged_df = df.merge(customer_stat, on=group_by_cols, suffixes=("",suff))
    
    # If multiple modes exist they are stored as a list -> handle them differently
    filter_single_mode = merged_df[merged_df[agg_col_stat].apply(lambda x: isinstance(x, np.int64))]
    filter_multi_mode = merged_df[merged_df[agg_col_stat].apply(lambda x: not isinstance(x, np.int64))]
    
    
    label_dist_single_mode = filter_single_mode[filter_single_mode[agg_col] != filter_single_mode[agg_col_stat]][["order", "label"]].drop_duplicates()["label"].value_counts()
    label_dist_multi_mode = filter_multi_mode[filter_multi_mode.apply(lambda x: x[agg_col] not in x[agg_col_stat], axis=1)][["order", "label"]].drop_duplicates()["label"].value_counts()
    
    label_dist = label_dist_single_mode + label_dist_multi_mode
    label_dist = label_dist/label_dist.sum()
    return label_dist


In [ ]:
group_difference_label_dist = create_user_stat_dist(train_df, ["customer"], "group", lambda x: pd.Series.mode(x))
size_difference_label_dist = create_user_stat_dist(train_df, ["customer", "group"], "size", lambda x: pd.Series.mode(x))

In [ ]:
label_dist =  train_df[["order", "label"]].drop_duplicates()["label"].value_counts()
label_dist = label_dist/label_dist.sum()

In [ ]:
dist_df = pd.DataFrame({"original": label_dist, "group_diff": group_difference_label_dist, "size_diff": size_difference_label_dist})
dist_df.index = ['no_return', 'partial_return', 'full_return']
dist_df.plot(kind="bar", rot=0)